In [1]:
import time
import datetime
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import DataLoader
from AlphaNet.Models import AlphaNet_LSTM_V1

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [2]:
data_path = "/home/wuwenjun/Data/"

loader1 = DataLoader()
loader1.load_data_from_file(alpha_name = "AlphaNet_8Input_corr_filter_Shift_1_Sequence_20_Transformer",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [03:09<00:00,  9.02it/s] 


COV_div_open_close_div_high_close_10_Shift_19  ...    target
timestamp           ticker                                                 ...          
2016-02-19 09:30:00 000001                                       0.000439  ...  0.432847
                    000004                                       0.002332  ... -0.169674
                    000005                                       0.002369  ...  0.731980
                    000006                                       0.002173  ...  1.232916
                    000008                                       0.002430  ... -0.137450
...                                                                   ...  ...       ...
2021-05-31 09:30:00 688698                                       0.000441  ... -0.042560
                    688777                                       0.000267  ... -0.447396
                    688788                                       0.000304  ... -0.597590
                    688819                                       0.000078  ...  0.134673
                    688981                                       0.000106  ... -0.320582

[4052504 rows x 1081 columns]

In [3]:
loader2 = DataLoader()
loader2.load_data_from_file(alpha_name = "AlphaNet_8Input_corr_filter_5dMoving_Shift_1_Sequence_20",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [03:09<00:00,  8.98it/s] 


COV_div_open_close_div_high_close_5_Shift_19  ...  DECAY_turnover_5_Shift_0
timestamp           ticker                                                ...                          
2016-02-05 09:30:00 000001                                      0.000743  ...                  0.307867
                    000004                                      0.002670  ...                  1.959540
                    000005                                      0.002917  ...                  3.499820
                    000006                                      0.002723  ...                  1.854653
                    000008                                      0.002700  ...                  0.810560
...                                                                  ...  ...                       ...
2021-05-31 09:30:00 688698                                      0.000259  ...                  1.978493
                    688699                                      0.000394  ...                  8.529500
                    688777                                      0.000537  ...                  3.888533
                    688788                                      0.000016  ...                  6.343567
                    688981                                      0.000114  ...                  2.607580

[4077016 rows x 1080 columns]

In [4]:
loader3 = DataLoader()
loader3.load_data_from_file(alpha_name = "AlphaNet_8Input_corr_filter_20dMoving_Shift_1_Sequence_20",end_date = "2022-01-01",data_path=data_path)

100%|██████████| 1705/1705 [03:03<00:00,  9.27it/s] 


COV_div_open_close_div_high_close_20_Shift_19  ...  DECAY_turnover_20_Shift_0
timestamp           ticker                                                 ...                           
2016-03-04 09:30:00 000001                                       0.000338  ...                   0.396974
                    000004                                       0.001981  ...                   2.153136
                    000005                                       0.002123  ...                   5.704872
                    000006                                       0.001678  ...                   3.678169
                    000008                                       0.001836  ...                   1.146375
...                                                                   ...  ...                        ...
2021-05-31 09:30:00 688699                                       0.001102  ...                   8.904934
                    688777                                       0.000242  ...                   3.232738
                    688788                                       0.000237  ...                   5.100587
                    688819                                       0.000177  ...                   2.762758
                    688981                                       0.000101  ...                   1.843993

[4011480 rows x 1080 columns]

In [6]:
final_df = pd.concat([loader1.feature_data,loader2.feature_data,loader3.feature_data],axis=1)
final_df

COV_div_open_close_div_high_close_10_Shift_19  ...  DECAY_turnover_20_Shift_0
timestamp           ticker                                                 ...                           
2016-02-05 09:30:00 000001                                            NaN  ...                        NaN
                    000004                                            NaN  ...                        NaN
                    000005                                            NaN  ...                        NaN
                    000006                                            NaN  ...                        NaN
                    000008                                            NaN  ...                        NaN
...                                                                   ...  ...                        ...
2021-05-31 09:30:00 688699                                            NaN  ...                   8.904934
                    688777                                       0.000267  ...                   3.232738
                    688788                                       0.000304  ...                   5.100587
                    688819                                       0.000078  ...                   2.762758
                    688981                                       0.000106  ...                   1.843993

[4104041 rows x 3241 columns]

In [7]:
final_df.dropna(axis=0,inplace=True)

In [8]:
final_df

COV_div_open_close_div_high_close_10_Shift_19  ...  DECAY_turnover_20_Shift_0
timestamp           ticker                                                 ...                           
2016-03-04 09:30:00 000001                                       0.000249  ...                   0.396974
                    000004                                       0.001573  ...                   2.153136
                    000005                                       0.001853  ...                   5.704872
                    000006                                       0.001297  ...                   3.678169
                    000008                                       0.001193  ...                   1.146375
...                                                                   ...  ...                        ...
2021-05-31 09:30:00 688689                                       0.000826  ...                   9.465556
                    688698                                       0.000441  ...                   3.065519
                    688777                                       0.000267  ...                   3.232738
                    688788                                       0.000304  ...                   5.100587
                    688981                                       0.000106  ...                   1.843993

[3960691 rows x 3241 columns]

In [9]:
AlphaNet.Data.convert_to_standard_daily_data_par(final_df,"ResLstm_8Input_corr_filter_Shift_1_Sequence_20",data_path)

100%|██████████| 1276/1276 [29:50<00:00,  1.40s/it]
